In [185]:
import pandas as pd
import unicodedata

from tqdm import tqdm
tqdm.pandas(desc="Progress")

In [2]:
import os
path = os.getcwd()
path

'/Users/ryanphung/Projects/Linguistics/chinese-hanviet'

In [36]:
df_chars = pd.read_csv('inputs/unihan.txt', sep='\t', keep_default_na=False)
df_chars

,character,language,value
0,U+3400,kCantonese,jau1
1,U+3400,kDefinition,(same as U+4E18 丘) hillock or mound
2,U+3400,kMandarin,qiū
3,U+3401,kCantonese,tim2
4,U+3401,kDefinition,"to lick; to taste, a mat, bamboo bark"
...,...,...,...
196650,U+30729,kHangul,은:N
196651,U+30EDD,kDefinition,biangbiang noodles
196652,U+30EDD,kMandarin,biáng
196653,U+30EDE,kDefinition,biangbiang noodles


In [39]:
df_hanviet_chars = df_chars[df_chars['language']=='kVietnamese'][['character','value']]
df_hanviet_chars['character'] = df_hanviet_chars['character'].map(lambda a: chr(int(a[2:], 16)))
df_hanviet_chars

,character,value
226,㑫,nộm
572,㓟,phay
577,㓠,chẻm
653,㓶,khía
1045,㕭,êu
...,...,...
195920,𪘵,răng
195996,𪚚,trong
196003,𪚤,trông
196032,𪛅,rùa


In [171]:
df_words = pd.read_csv('inputs/leeds.txt', sep=' ', names=['rank','frequency','word'], skiprows=[0,1,2,3])

In [259]:
def get_hanviet(word):
    result = []
    
    for c in word:
        df_of_c = df_hanviet_chars[df_hanviet_chars['character'] == c]
        if df_of_c.empty:
            return
        c_hv = df_of_c['value'].values[0].split(' ')
        if (len(result) == 0):
            result = c_hv
        else:
            row = result
            result = []
            for i in row:
                for j in c_hv:
                    result.append('-'.join([i, j]))
    
    return ','.join(result)
    
print(get_hanviet('利害'))
print(get_hanviet('我们'))
print(get_hanviet('看看'))

lợi-hại
None
trầu-trầu,trầu-khan,khan-trầu,khan-khan


In [260]:
df_words['hanviet'] = df_words.progress_apply(lambda x:get_hanviet(str(x['word'])), axis=1)
df_words


my bar!: 100%|██████████| 50000/50000 [01:22<00:00, 608.32it/s]


,rank,frequency,word,hanviet
0,1,66293.28,，,None
1,2,45402.97,的,đích
2,3,14216.88,了,liễu
3,4,12935.94,我,ngã
4,5,11879.45,一,nhất
...,...,...,...,...
49995,49996,0.63,陕西人,None
49996,49997,0.63,封建制度,phong-kiến-chế-độ
49997,49998,0.63,牛郎织女,None
49998,49999,0.63,着眼点,None


In [261]:
df_words_hanviet = df_words[df_words['hanviet'].notnull()]
df_words_hanviet

,rank,frequency,word,hanviet
1,2,45402.97,的,đích
2,3,14216.88,了,liễu
3,4,12935.94,我,ngã
4,5,11879.45,一,nhất
5,6,9854.95,在,tại
...,...,...,...,...
49984,49985,0.63,成林,thành-lâm
49988,49989,0.63,交合,giao-hợp
49989,49990,0.63,背出,bối-xuất
49992,49993,0.63,十一二,thập-nhất-nhì


In [263]:
df_words_hanviet.to_csv('outputs/hanviet.tsv', sep='\t', index_label=['no'])